# Theano Basics 2

- Default Values
- Shared Variables
- Random Variables
- Example: Logistic Regression

## Default Values

Use the [`theano.function.Param`](http://deeplearning.net/software/theano/library/compile/function.html#function.Param) class to **attach informations to function inputs**. 

For example, to give a default value of 1 for `y`, create a `Param` instance with its `default` field set to 1 :

```
theano.Param(y, default=1)
```

Note :
- use the [`theano.function.Out`](http://deeplearning.net/software/theano/library/compile/function.html#function.Out) class to attach informations to function outputs
- use the [`theano.compile.io.In`](http://theano.readthedocs.org/en/latest/library/compile/io.html#io.In) class to attach informations to function inputs (similar to `Param`)

#### Python version

In [ ]:
def f(x, y=1):
    return x + y

#### Theano version

In [ ]:
import theano
import theano.tensor as T
import theano.Param as Param

x, y = T.dscalars('x', 'y')
z = x + y
f = theano.function([x, Param(y, default=1)], z)

In [ ]:
print f(33)
print f(33, 2)
print f(34)

## Shared Variables

Use [`theano.shared(value, name=None)`](http://theano.readthedocs.org/en/latest/library/compile/shared.html#module-shared) to create a `SharedVariable` instance.

- **`value`** : value to associate with this variable (a new container will be created)
- **`name`** *(None or str)* : name for this variable
- **`type`** : Type of this Variable


A `SharedVariable` is a Variable with an internal value that is **shared between *all* functions that use**.
The value can be accessed and modified by the `.get_value()` and `.set_value()` methods.

In order to use a `SharedVariable` in a function, provide the `updates` option with a pair (SharedVariable, new SharedVariable expression) giving informations on what `SharedVariable` to use and how to update it.

In [ ]:
count = theano.shared(0)
inc = T.iscalar('inc')

add_inc = count + inc
subtract_inc = count - inc

incrementor = theano.function([inc], count, updates=[(count, add_inc)])
decrementor = theano.function([inc], count, updates=[(count, subtract_inc)])

In [ ]:
count.get_value()

incrementor(2)
count.get_value()

decrementor(4)
count.get_value()

incrementor(12)
count.get_value()

## Random Variables

The way to think about putting randomness into Theano's computations is to put random variables in your graph. Theano will allocate a NumPy [`RandomStreams`](http://theano.readthedocs.org/en/latest/library/tensor/shared_randomstreams.html#libdoc-tensor-shared-randomstreams) object (a random number generator) for each such variable, and draw from it as necessary. We will call this sort of sequence of random numbers a random stream. Random streams are at their core shared variables, so the observations on shared variables hold here as well.

In the example below : 

- `rng` : random number generator.
- `rv_u` : random stream (shared variable) of 2x2 matrices of draws from a uniform distribution.
- `rv_n` : random stream (shared variable) of 2x2 matrices of draws from a normal distribution.
- `f()` : returns **a new random uniform number** every time it is called. The internal state of the random number generator is automatically updated, so we get different random numbers every time.
- `g()` : returns **the same random normal number** every time it is called. The internal state of the random number generator is not affected when we add the extra argument `no_default_updates=True`.

In [ ]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function

rng = RandomStreams(seed=234)

rv_u = rng.uniform((2,2))
rv_n = rng.normal((2,2))

f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)    #Not updating rv_n.rng

In [ ]:
print f()
print f()
print g()
print g()

Note that a random variable is **drawn at most once** during any single function execution. So the `nearly_zeros()` function is guaranteed to return approximately 0 (except for rounding error) even though the `rv_u` random variable appears three times in the output expression.

In [ ]:
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)
print nearly_zeros()

# Logistic Regression